In [54]:
%load_ext autoreload
%autoreload 2
from importlib import reload

from collections import defaultdict

import numpy

import random
from matplotlib import pyplot as plt
from random import shuffle, choice, randint

import math
import keras
import tensorflow

from NN_utils import BigTable, WeightClip

import pickle
import zlib

def logistic(x, b,off):
    z = b*(x-off)
    return numpy.exp(z)/(1+numpy.exp(z))

def pr_to_spread(p, comps=1, as_A_and_D=True):
    per_comp_p = p**(1/(comps))
#     print("p         ", p)
#     print("per comp p", per_comp_p)
#     spread = -numpy.log((1.0/per_comp_p)-1.0)
    inv_sigmoid = lambda pr : ( -numpy.log((1/pr) -1) )
    spread = inv_sigmoid(per_comp_p)
#     print("spread    ", spread)
    if as_A_and_D:
        a = spread/2.0
        d = -spread/2.0
        return a,d
    else:
        return spread

print("started")


from sklearn.svm import SVR


def calc_probs(s,q):
    zmask = numpy.isclose(q,-10).astype(int)
    diff = s-q
    prs = 1.0/(1.0+ numpy.exp(-diff))
    prs = numpy.maximum(zmask,prs)
    pr = numpy.prod(prs, axis=-1).reshape(len(q))
    return pr

def calc_probs_from_embs(students,questions):
#     students2 = numpy.repeat(students, len(questions), axis=0)
#     questions2 = numpy.tile(questions, (len(students),1))
#     zmask = numpy.isclose(questions2,-10).astype(int)
#     diffs = students2-questions2
#     prs = 1.0/(1.0+ numpy.exp(-diffs))
#     prs = numpy.maximum(zmask,prs)
#     probs2 = numpy.prod(prs, axis=1).reshape(len(students), len(questions))
    probs2 = []
    for s in students:
        pr = calc_probs(s,questions)
        probs2.append(pr)
    return numpy.array(probs2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
started


In [55]:
def calc_arr_arr_err(split, real_wgts, pred_wgts, max_iter=10):
    from scipy.spatial.distance import cosine
# pred_wgts = numpy.round(pred_wgts,1)

    print(split, real_wgts.shape, pred_wgts.shape, max_iter)

    out_cols = [None] * len(real_wgts.T)
    curr_sel = None
    curr_ix = None
    n_iters = 10
    chosen = None
    curr_real_ix = None
    
    indices = range(len(real_wgts.T))

    min_total_err = math.inf
    best_dis = math.inf
    for i in range(max_iter): #len(indices)**2):
        real_used = set()
        pred_used = set()
        while len(pred_used) < len(indices):
            curr_err = math.inf
            curr_cos = math.inf
            for rix in numpy.random.permutation(indices):
                if rix in real_used:
                    continue
                real_col = real_wgts.T[rix]
                for cix in numpy.random.permutation(indices):
                    if cix in pred_used:
                        continue
                    pred_col = pred_wgts.T[cix]
                    pred_col = pred_col #* pred_q_col
                    err = numpy.mean(numpy.abs( pred_col - real_col))
                    
                    if err < curr_err:
                        curr_sel = pred_col
                        curr_err = err
                        curr_cos = 0#cosine(pred_col, real_col)
                        curr_ix = cix
                        curr_real_ix = rix
            real_used.add(curr_real_ix)
            pred_used.add(curr_ix)
            out_cols[curr_real_ix] = curr_sel
        out_col_arr = numpy.array(out_cols).T
        total_err = numpy.mean(numpy.abs( out_col_arr - real_wgts ))
        
        dis = 0
        mean_ll = numpy.mean( out_col_arr - real_wgts )
        if total_err < min_total_err:
            min_total_err = total_err
            total_q_err = numpy.mean(numpy.abs( out_col_arr[0:split] - real_wgts[0:split] ))
            total_s_err = numpy.mean(numpy.abs( out_col_arr[split:] - real_wgts[split:] ))
            best_ll = mean_ll
            chosen = out_col_arr
            best_dis = dis
    return chosen, min_total_err, total_q_err, total_s_err, mean_ll, best_dis


In [56]:
import copy
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
def plot_items(pred_list, real_items, s_offset):
    if real_items.shape[1]<2:
        print("real_items is only 1 component wide .. needs to be >1 to plot on a PCA graph")
        return None
    elif real_items.shape[1]==2:
        print("2 comps, so no dim reduc")
        tx=None
    else:
        tx = PCA(n_components=2)
#         tx = TSNE(n_components=2)
    fyrst = True
    Cs = []
    C_labs = []
    pred_list = numpy.array(pred_list)
    print("pred list shape", pred_list.shape)
    print("real items shape", real_items.shape)


    fitted_pred_list = []
#     offset = numpy.median(pred_list[(pred_list>0.1)], axis=0) - numpy.median(real_items[(real_items>0.1)], axis=0)
#     real_mean = numpy.min(real_items[real_items > 0.1])
#     offset = numpy.min(pred_list[pred_list > 0.1]) - real_mean
#     print("real mean", real_mean)
#     print("offset", offset)
    
    m = len(real_items)
    cols = list(range(m))
    shuffle(cols)
    
    xmeans = numpy.zeros(m)
    ymeans = numpy.zeros(m)
    pairs = defaultdict(list)
    iter = 0
    
    cp_real = copy.copy(real_items)
    cp_real[cp_real < 1] = numpy.nan
    r_offset=numpy.nanmedian(cp_real, axis=0)
    
    itemz_2 = real_items
    n = len(real_items)
    
    for opreds in pred_list:
        preds = copy.copy(opreds) #- s_offset[iter] + r_offset
        split = 0
        
        items_chosen, min_total_err, total_q_err, total_s_err, mean_ll, best_cos_dis = calc_arr_arr_err(0, real_items, preds, max_iter=10)

        itemz_pred = items_chosen
        print(itemz_pred)
#         itemz_pred = numpy.maximum(itemz_pred,0)
        fitted_pred_list.append(itemz_pred)
        
#         itemz = real_items #- offset
#         print(numpy.min(itemz), numpy.mean(itemz), numpy.max(itemz))
#         itemz = numpy.maximum(itemz,0)
#         print(numpy.min(itemz), numpy.mean(itemz), numpy.max(itemz))
#         if itemz_2 is None:
#             itemz_2 = numpy.concatenate([real_itemz, itemz_pred], axis=0)
#         else:
        itemz_2 = numpy.concatenate([itemz_2, itemz_pred], axis=0)

    if tx:
        itemz_2 = tx.fit_transform(itemz_2)
#         itemz_2 = numpy.concatenate([itemz, itemz_pred], axis=0)
#         if fyrst:
#         itemz_2 = tx.fit_transform(itemz_2)
#             fyrst = False
#         else:
#             itemz_2 = tx.transform(itemz_2)

    from sklearn.cluster import KMeans
    iter=0
    fig = plt.gcf()
    fig.set_size_inches(10, 10)
    
    for opreds, itemz_pred in zip(pred_list, fitted_pred_list):
        n = len(itemz_pred)
        km = KMeans()
        km.fit(itemz_pred)
        cluster_labels = km.predict(itemz_pred)
        print(cluster_labels)
        
        C = []
        for l in set(cluster_labels):
            cluster = list(numpy.where(cluster_labels==l)[0])
            print("X", cluster)
            C.append(cluster)
        Cs.append(C)
        C_labs.append(cluster_labels)
                
#         NUM_COLORS = 100
#         cm = plt.get_cmap('gist_rainbow')
#         fig.gca().set_color_cycle([cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
        print(type(itemz_2))
        minix=n*(iter+1)
        maxix=n*(iter+1)+n
        
        #i=0 -> 100,199
        #i=2 -> 200,299
        
        print("no pts=",n," indices=", minix, maxix)
        print("shape of itemz_2", itemz_2.shape)
        fig.gca().scatter(itemz_2[minix:maxix,0], itemz_2[minix:maxix,1], alpha=0.7, c=numpy.array(cols), cmap=plt.get_cmap('nipy_spectral'))
        j=0
        for j in range(n):
            x,xh,y,yh = itemz_2[j+(n*iter),0], itemz_2[j+(n*iter+n) ,0], itemz_2[j+(n*iter),1], itemz_2[j+(n*iter+n),1]
#             fig.gca().plot([x,xh],[y,yh],  color="#aaaaaa80")
            xmeans[j] += xh
            ymeans[j] += yh
            pairs[iter].append((xh, yh))
        iter+=1
        
    for j in range(n):
        fig.gca().annotate(j, (itemz_2[j,0], itemz_2[j,1]))
        
    fig.gca().scatter(itemz_2[0:n,0], itemz_2[0:n,1], c="b", zorder=10, alpha=0.5)
#     fig.gca().axhline(y=1e-6, linestyle="--")
#     fig.gca().axvline(x=1e-6, linestyle="--")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    
    xmeans = xmeans / len(pred_list)
    ymeans = ymeans / len(pred_list)
    

    if len(pred_list)>=1:
        for it in range(len(pred_list)):
            xhyh_pairs = pairs[it]
            for j,hat_pair in enumerate(xhyh_pairs):
                xh,yh = hat_pair
                x,y = itemz_2[j,0], itemz_2[j,1]
                mux = xmeans[j]
                muy = ymeans[j]
#                 fig.gca().scatter(xh, yh, alpha=0.7, c=plt.get_cmap('nipy_spectral')(cols[j]))
#                 fig.gca().scatter(mux,muy, c="#888888ff", marker="*", zorder=10)
#                 fig.gca().plot([mux,xh],[muy,yh],color="#aaaaaa80", linestyle="--")
#                 fig.gca().plot([mux,x],[muy,y],color="#888888dd", linestyle="-")
                fig.gca().plot([x,xh],[y,yh],color="#aaaaaa80", linestyle="--")
        
    plt.show()
    print("len Cs", len(Cs))
    from sklearn.metrics.cluster import adjusted_rand_score
    rands = []
    for ix in range(len(Cs)):
#         print(ix)
        for jx in range(len(Cs)):
#             print(jx)
            if ix!=jx:
#             print(Cs[ix], Cs[jx])
#                 print("VI:", ix,jx, varinfo(Cs[ix],Cs[jx]))
                a_rand = adjusted_rand_score(C_labs[ix], C_labs[jx])
                print("Rand:", a_rand)
                rands.append(a_rand)
    print("Mean rand score =", numpy.mean(rands), numpy.std(rands))

# qws = qn_table.get_weights()[0]
# qws2 = qn_table2.get_weights()[0]

In [57]:
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score
def generate_and_train(n_students, n_questions, qz,sz,pfz, vqz,vsz,vpfz, w, n_factors, min_active, max_active, nn_mode=None, loss_mode=None, class_weights=None):
    btm = 0
    top = math.sqrt(.1/w)
#     init= (btm,top)
#     init = math.sqrt(.5/w)
    init_s = (0,1)
    init_q = (0,1)
    
#     1-p + pq = s
#     q=0.3 : 1-p + p/3 = s
#           : 3-3p + p = s
#           : p = (3-s)/2
        
    if nn_mode=="COND":
        percompp = .5**(1/w)
        print("percompp", percompp)

        s_table =  BigTable((n_students, w), 0,1, init_hilo= percompp )#, regulariser=regularizers.l2(10e-6))
        qn_table = BigTable((n_questions, w), 0,1, init_hilo= percompp )#, regulariser=regularizers.l1(10e-6))
    elif nn_mode=="MXFN":
        init = math.sqrt(.5/w)
        print("MXFN init'n")
        print(init)
        print(init*init*w)
        s_table =  BigTable((n_students, w), -math.inf, math.inf, init_hilo= init) #, regulariser=regularizers.l2(10e-6))
        qn_table = BigTable((n_questions, w), -math.inf, math.inf, init_hilo= init) #, regulariser=regularizers.l1(10e-6))
    elif nn_mode=="MLTM":
        sp = pr_to_spread(.5, w, as_A_and_D=False)
        print("sp is ",sp)
        s_table =  BigTable((n_students, w), -math.inf, math.inf, init_hilo= 0) #, regulariser=regularizers.l2(10e-6))
        qn_table = BigTable((n_questions, w), -math.inf, math.inf, init_hilo= -sp) #, regulariser=regularizers.l1(10e-6))        
    else:
        s_table =  BigTable((n_students, w), -math.inf, math.inf, init_hilo= 0) #, regulariser=regularizers.l2(10e-6))
        qn_table = BigTable((n_questions, w), -math.inf, math.inf, init_hilo= 0) #, regulariser=regularizers.l1(10e-6))        
                
    from keras.layers import Embedding
    from keras.constraints import NonNeg, MinMaxNorm
    from keras.initializers import RandomNormal, RandomUniform
    
#     wc=WeightClip(0,1)
    
#     q_gates = None #Embedding(n_questions,w, input_length=1, embeddings_initializer=RandomUniform(minval=0, maxval=1, seed=None), embeddings_constraint=wc)
#     qn_table = Embedding(n_questions,w, input_length=1, embeddings_initializer=RandomNormal(mean=6, stddev=0.3))
#     s_table = Embedding(n_students,w, input_length=1, embeddings_constraint=WeightClip(0,math.inf), embeddings_initializer=RandomNormal(mean=6, stddev=0.3))
    
    from keras.optimizers import Adam
    from keras.callbacks import EarlyStopping, LambdaCallback
    
#     vqz=[]
    if len(vqz)>0:
        lozz="val_mean_absolute_error"
        val_dat= [[vqz,vsz], vpfz]
    else:
#     if True:
        lozz="mean_absolute_error"
        val_dat=None
    
    fiftiez = numpy.zeros_like(pfz) + .50
    for _ in range(1):
#         es = EarlyStopping(monitor="loss", restore_best_weights=True, patience=10)
#         m = generate_qs_model(qn_table, s_table, Adam(lr=0.001))
#         h = m.fit(x=[qz,sz], y=numpy.array(fiftiez).reshape(-1,1), batch_size=len(pfz), shuffle=True, epochs=10000, verbose=1, callbacks=[es])
#         wz = m.get_weights()
        m = generate_qs_model(qn_table, s_table, Adam(), _mode=nn_mode, loss=loss_mode)
#         m.set_weights(wz)

        tr_predz = m.predict([qz,sz])
        print(tr_predz)
        tr_predz = (m.predict([qz,sz])[:,0] > 0.5)
#         for vs,vq,tp,pp in zip(vsz,vqz,predz, vpfz):
#             print(vs,vq,"-",tp,pp)
        print("PRE-TR AVG  = ", numpy.mean(tr_predz))
    
        if len(vqz)>0:
            v_predz  = (m.predict([vqz,vsz])[:,0] > 0.5)
            print("PRE-TR VAVG = ", numpy.mean(v_predz))

        es = EarlyStopping(monitor=lozz, restore_best_weights=True, patience=10)
        

        prs_model = Model(inputs=m.input,
                                outputs=m.get_layer("Pr_sigmoid1").output)
#         prs_output = prs_model.predict([qz,sz])
        

        
        hazard_model = Model(inputs=m.input,
                                outputs=m.get_layer("hazard").output)
#         intermediate_output = intermediate_layer_model.predict([qz,sz])
        
        print_prs = LambdaCallback(on_epoch_end=lambda batch, logs: 
#                                        print(numpy.min(intermediate_layer_model.predict([qz,sz])),
#                                              numpy.max(intermediate_layer_model.predict([qz,sz]))))
                                       print(prs_model.predict([qz[0:10],sz[0:10]])))

        print_hazard = LambdaCallback(on_epoch_end=lambda batch, logs: 
                                       print(hazard_model.predict([qz[0:10],sz[0:10]])))

        
        
        _bs = len(pfz)
#         _bs = 1000
        for _ in range(1):
            h = m.fit(x=[qz,sz], y=pfz, batch_size=_bs, class_weight=class_weights, shuffle=True, epochs=10000, verbose=1, callbacks=[es], validation_data=val_dat)
        tr_predz = m.predict([qz,sz])
        if len(vqz)>0:
            v_predz  = m.predict([vqz,vsz])

#         print("TR AVG = ", numpy.mean(tr_predz))
#         print("TR R2  = ", r2_score(pfz, tr_predz))
#         print("TR MAE = ", mean_absolute_error(pfz, tr_predz))
#         print("TR ACC = ", accuracy_score((pfz>0.5), (tr_predz>0.5)))
#         print("TR AGT = ", accuracy_score([random.random() < p for p in pfz], [random.random() < p for p in tr_predz]))
        
#         if val_dat:
#             print("VA AVG = ", numpy.mean(v_predz))
#             print("VA R2  = ", r2_score(vpfz, v_predz))
#             print("VA MAE = ", mean_absolute_error(vpfz, v_predz))
#             print("VA ACC = ", accuracy_score((vpfz>0.5), (v_predz>0.5)))
#             print("VA AGT = ", accuracy_score([random.random() < p for p in vpfz], [random.random() < p for p in v_predz]))

#             for ent,hat in zip(vpfz, v_predz):
#                 print(ent)
#                 print(hat)
#                 print(numpy.sum(hat))
#                 print("~~~~")

    from sklearn.metrics import classification_report
    print(classification_report((pfz>0.5), (tr_predz>0.5)))
    if len(vqz)>0:
        print(classification_report((vpfz>0.5), (v_predz>0.5)))
            
#     h = m.fit(x=[qz,sz], y=pfz.flatten(), batch_size=32, shuffle=True, epochs=1000, verbose=1, callbacks=[es], validation_data=[[vqz,vsz], vpfz])
    return s_table, qn_table, m, h


In [58]:
def stitch_n_split(_pairs, sts, qns, realise=True, rpt=False):
    out_w = 1
    max_fails = out_w -1
    if realise:
        if rpt:
            _counts = defaultdict(int)
            _matches=[]
            _pfz=[]
            _sz, _qz = [],[]
            cache = defaultdict(list)
            for (vi,mi) in _pairs:
                cache[vi].append(mi)
            
            for vi in cache:
                s = sts[vi]
                prs = calc_probs(s, [qns[k] for k in cache[vi]])  
#                 print("shape of prs", prs.shape)
                for mix,mi in enumerate(cache[vi]):
#                 pr = _probs[vi,mi]
                    pr = prs[mix]
                    rnd = random.random()
                    i = 0
                    zs = numpy.zeros(out_w)
                
                    while rnd > pr and i<out_w:
                        i += 1
                        rnd = random.random()
                    
                    if max_fails > 0:
                        zs[min(i, max_fails)] = 1
                    else:
                        zs[0] = int(i==0)
            
                    _counts[i] += 1

                    _pfz.append(zs)
    #                 _pfz.append(i)
                    _sz.append(vi)
                    _qz.append(mi)
            print("probs calced")
        else:
            _prob_list =  numpy.array([calc_prob(sts[vi],qns[mi]) for (vi,mi) in _pairs])
            _pfz = (numpy.random.random(len(_prob_list)) < _prob_list).astype(int)
    #     _pfz = (0.5 < _prob_list).astype(int)
            _matches = ( numpy.round(_prob_list) == _pfz).astype(int)
        print("realisation complete")
    else:
        cache = defaultdict(list)
        for (vi,mi) in _pairs:
            cache[vi].append(mi)

        _prob_list=[]
        for vi in cache:
            q_keys = [k for k in cache[vi]]
            s = sts[vi]
            prs = calc_probs(s, [qns[k] for k in q_keys])  
            for mix,mi in enumerate(q_keys):
                pr = prs[mix]
                _prob_list.append(pr)
                
        _prob_list =  numpy.array(_prob_list)
        _pfz = _prob_list
        _sz = [p[0] for p in _pairs]
        _qz = [p[1] for p in _pairs]

#     print(_pfz)
#     _pfz = numpy.array([probs[vi,mi] for (vi,mi) in _pairs])

#     print(_matches)
#     print(numpy.sum(_matches), "correctly labelled out of", len(_matches), "%=", numpy.sum(_matches)/len(_matches))
    if not rpt:
        _sz = [p[0] for p in _pairs]
        _qz = [p[1] for p in _pairs]
    print("sns complete")
    
#     one_c = _counts[1]
#     for k in _counts:
#         c = _counts[k]
#         _counts[k] = one_c / c
    
    return numpy.array(_pfz), _sz, _qz, None

In [59]:
gen_m_cache = pickle.load(open("generators.p", "rb"))

In [60]:
# gen_m_cache = {}

In [61]:
from sklearn.metrics import r2_score, mean_absolute_error

def report(n_factors, min_active, max_active, emb_w, nn_mode, loss_mode, sws_list, qws_list, model_list, real_stu_list, real_que_list, test_datasets, params_list, spars_list, compare=False):
    
    tot_sqerr = 0
    mean_err_list = []
    mean_std_list = []
    mean_hit_list = []
    
    print("*****")
    print(nn_mode, loss_mode)
#     print("*****")
    print(len(sws_list), len(qws_list), len(model_list), len(real_stu_list), len(real_que_list), len(test_datasets), len(params_list))
    
    for sw,qw,m,stz,qnz,tt_pairs, params, spars in zip(sws_list, qws_list, model_list, real_stu_list, real_que_list, test_datasets, params_list, spars_list):
        tw,a1,a0,trbal,vbal,agt = params
        
        print("params:", n_factors, min_active, max_active, emb_w, "/", tw,a1,a0, "(", trbal,vbal,agt,") [",spars,"]")
        
        err_list = []
        true_err_list = []
        hit_list = []
    #     for six,qix in numpy.sort(tt_pairs, axis=0):
    
        true_pz = []
        pred_pz = []
        for six, qix in tt_pairs:
    #         print(six, qix)
    #     print("\n------\n")
    #     continue
    #     if False:
            tq = qnz[qix,:]
            ts = stz[six,:]
            qrow = qw[qix, :]
            srow = sw[six, :]
#             print(qrow)
    #         print("raw",tq,ts)
    #         print("dif",ts-tq)
    #         print(numpy.prod(logistic(ts-tq,1,0)))
#             if rasch:
            true_p = float(calc_probs_from_embs(ts.reshape(1,-1),tq.reshape(1,-1)))
#                 dif = ts-tq
#                 true_ps = 1.0 / (1.0 + numpy.exp(-dif))
#                 true_p = numpy.prod(true_ps)
#             else:
#                 true_p = numpy.prod((1-tq)+(ts*tq))
            pred_p = m.predict([[qix],[six]]).flatten()[0]
    
            if compare:
                print(six,qix, ":", true_p, pred_p)
    
            true_pz.append(float(true_p))
            pred_pz.append(float(pred_p))
    #         pred_p = random.random()
    
            mae = numpy.abs(true_p - pred_p)
#             print(true_p, float(pred_p), "err:", float(mae))

            err = true_p - pred_p

            true_err_list.append(err)
            err_list.append(mae)
            good_guess = int(numpy.round(true_p))==int(numpy.round(pred_p))
            hit_list.append(int(good_guess))
    #         sqerr = numpy.power(true_p - pred_p, 2)

#             print(six, qix, ":", srow, qrow)
#             print("-->", pred_p, true_p, " ... ", good_guess)

        print("R2 = ", r2_score(true_pz, pred_pz))
        print("MAE = ", mean_absolute_error(true_pz, pred_pz))
        numpy.set_printoptions(precision=3)
    #     print("Mean sq err {}:".format(qrow.shape), numpy.sqrt(numpy.mean(err_list)))
    
        plt.hist(true_pz, alpha=0.5)
        plt.hist(pred_pz, alpha=0.5)
        plt.show()
        
        plt.hist(numpy.array(true_err_list).flatten(), alpha=0.5)
        plt.show()
        
        mean_err_list.append(numpy.mean(err_list))
        mean_std_list.append(numpy.std(err_list))
        mean_hit_list.append(numpy.mean(hit_list))
    #     print(sum(hit_list), len(hit_list), sum(hit_list)/len(hit_list))

    # print(mean_err_list)
    # print(mean_std_list)
    # print(mean_hit_list)
    # print(params_list)
    print(len(stz),"x",len(qnz))
#     for e,s,acc,params in zip(mean_err_list, mean_std_list, mean_hit_list, params_list):
#         print("acc=",acc)
#         print("mae=",e,"sig=",s)
#         print(params)
#     print("aggregated:")
    print(numpy.median(mean_hit_list), numpy.std(mean_hit_list), "/", numpy.median(mean_err_list), numpy.median(mean_std_list))
    print(numpy.median(mean_err_list), numpy.mean(mean_err_list))
# report(n_factors, min_active, max_active, emb_w, nn_mode, loss_mode, sws_list, qws_list, model_list, real_stu_list, real_que_list, test_datasets, params_list)

In [62]:
data_cache = {}
pfz = None
amx1 = None
qws1 = None

In [63]:
# def render_deets(ss,qs, n=10):
#     for six,s in enumerate(ss[0:n]):
#         for qix, q in enumerate(qs[0:n]):
#             zmask = numpy.isclose(q,-10).astype(int)
#             logits = s - q
#             prs = 1.0 / (1.0+numpy.exp(-logits))
#             prs = numpy.maximum(zmask,prs)
#             p = numpy.prod(prs)
#             print(six,qix,":")
#             print("SSSS")
#             print(s)
#             print("QQQQ")
#             print(q)
#             print(logits, "->", prs, "=>", p)
# render_deets(students2, questions)

In [64]:
##### tw should be ~U[0.5, 3.5]
#sw should be ~N[0, sd] with sd ~U[1, 3.5]
#a0 should be ~U[-0.5, 1]
#missing proportion should be ~U[0, 0.3]

# from tensorflow.random import set_seed



from sklearn.model_selection import train_test_split

explore_mode = False

reportz=[]

# factors_master = [(10,1,5)]
factors_master = [(100,1,5)]
w_list = [100]
factors_list = [ m+(w,) for m in factors_master for w in w_list ]

# nn_modes = ["MLTM","COND","MXFN"]
nn_modes = ["DEEP"]
loss_modes = ["MSE"]
# sq_nums = [(int(1000*(1.4)**3), int(150*(1.4)**3))]
# sq_nums = [(int(200*(1.41)**4), int(200*(1.41)**4))]
sq_nums = [(10000, 1000)]

# student_staminas = [0.01, 0.1, 0.5, 0.75, 1.0]

spars_list = [.25] # [0.01, 0.05, 0.25, 0.5, 0.75, 1.0]

data_to_run = [5, ]
bal = .5


for (n_students, n_questions) in sq_nums:
    print("{} students, {} questions".format(n_students, n_questions))
    for nn_mode in nn_modes:
        for loss_mode in loss_modes:
            for (n_factors, min_active, max_active, emb_w) in factors_list:
                for spars in spars_list:
                    
                    tup = ((n_factors, min_active, max_active), (10000, 70))
                    if tup in gen_m_cache:
                        (gen_m, history, best_dims, best_mse) = gen_m_cache[tup]
                    else:
                        print(gen_m_cache.keys())
                        raise Exception("Genny not found for",tup)

#                     set_seed(666)
                    numpy.random.seed(666)

                    pred_list  = []
                    model_list = []
                    sparss     = []
                    sws_list   = []
                    qws_list   = []
                    real_stu_list = []
                    real_que_list = []
                    params_list   = []
                    test_datasets = []
                    for a in data_to_run:
                        
                        fname = "MLTM_10000_1000_(100_1_5)_sp{:03d}_{}".format(int(spars*100), a)

                        (tw,a0,a1, students_temp, qz_temp) = pickle.load(open("./synth_data/MLTM_10000_1000_(100_1_5)_{}.p".format(a), "rb"))
                        print("loaded dataset",a,":", a0,a1,tw)
                              
                        from keras.models import load_model
                        m = load_model("./models/" + fname, custom_objects={'WeightClip': WeightClip})

                        n_students = 10000
                        n_questions = 1000
                        students2 = students_temp[0: n_students]
                        questions = qz_temp[0: n_questions]
                        print(n_students, n_questions)

                        slist = list(range(len(students2)))
                        qlist= list(range(len(questions)))
                        tr_pairs = []
                        for vi in slist:
                            for mi in qlist:
                                tr_pairs.append((vi,mi))
 
                        osz,oqz,opfz, vsz, vqz, vpfz, tt_pairs = pickle.load(open("./train_test_pairs/"+fname+".p".format(a), "rb"))
                        print("loaded o/v/t pairs from file")
                        Apfz, Asz, Aqz, _ = stitch_n_split(tr_pairs, students2, questions, realise=False, rpt=True)


#                         if explore_mode:
#                             plt.hist(numpy.array(pfz).flatten(), alpha=0.5)
#                             plt.title("pfz")
#                             plt.show()

                        print(n_students, n_questions)
                        probs = numpy.zeros((n_students, n_questions))
                        phats = numpy.zeros((n_students, n_questions))
                        for s,q,pf in zip(Asz,Aqz,Apfz):
                            probs[s,q] = pf
                            
                        maske = numpy.zeros((n_students, n_questions), dtype=int)
                        for s,q,pf in zip(osz, oqz, opfz):
                            maske[s,q] = 1
                        
                        for s,q,phat in zip(Asz,Aqz, m.predict([Aqz,Asz])):
                            phats[s,q] = phat
                    
#                         sortargs = numpy.argsort(numpy.median(probs, axis=1), axis=0)
#                         sortargs = numpy.argsort(probs, axis=0)
                        sortargs = range(probs.shape[0])
                        tx = PCA(n_components=2)
                        mean_probs = []
                        for prob_row in probs:
                            mean_probs.append(numpy.mean(prob_row))

#                             print(numpy.round(prob_row,2), numpy.mean(prob_row))
                        mean_proficiencies = []
                        for s_row in students2:
                            mean_proficiencies.append(numpy.mean(s_row))

                
                        for prob_row, s_row in zip(probs, students2):
                            numpy.mean(prob_row)
                
                        import scipy
                        from sklearn.preprocessing import minmax_scale, scale

#                         alphaz = numpy.mean(students2, axis=1)
                        
                        sts_scaled = minmax_scale(students2)
                        #alphaz = numpy.sqrt(numpy.sum(students2**2, axis=1))
#                         sts_txd = tx.fit_transform(sts_scaled)
#                         plt.scatter(x=sts_txd[:,0], y=sts_txd[:,1], c=alphaz)
#                         plt.show()

                        p_alphaz = numpy.mean(probs, axis=1)
                        ph_alphaz = numpy.mean(phats, axis=1)
                        s_alphaz = numpy.mean(students2, axis=1)
    
#                         probs_txd = tx.fit_transform(probs[sortargs,:])
                        probs_txd = tx.fit_transform(probs)
                        plt.scatter(x=probs_txd[:,0], y=probs_txd[:,1], c=s_alphaz)
#                         plt.scatter(x=students2[:,0], y=students2[:,1], c=s_alphaz)
                        plt.show()
                        
                        plt.scatter(x=probs_txd[:,0], y=probs_txd[:,1])
#                         plt.scatter(x=students2[:,0], y=students2[:,1], c=p_alphaz)
                        plt.show()
    
    
                        phats = numpy.transpose(phats)
                        maske = numpy.transpose(maske)
                        probs = numpy.transpose(probs)
                
#                         probs = minmax_scale(probs)
#                         phats = minmax_scale(phats)

#                         import matplotlib.pyplot as plt
#                         import numpy as np

#                         meanz = numpy.median(probs, axis=0)
#                         print("meanz shape", meanz.shape)
#                         sorth = numpy.argsort(meanz)
                        sorth = range(probs.shape[1])
#                         a = np.random.random((16, 16))
                        plt.figure(figsize=(8,8))
                        plt.imshow((probs * maske) + (probs/5), cmap='hot', interpolation='nearest')
#                         plt.imshow(maske, cmap='hot', interpolation='nearest')
                        plt.show()

                        plt.figure(figsize=(8,8))
                        plt.imshow(phats, cmap='hot', interpolation='nearest')
                        plt.show()                 
                        
                        
                        nn = 500 #len(probs)
                        concd = numpy.concatenate((probs[0:nn,:], phats[0:nn,:]))
                        plt.figure(figsize=(10,10))
#                         tx = TSNE(n_components=2)
                        pts_txd = tx.fit_transform(concd)
                        x,y = pts_txd[:nn,0], pts_txd[:nn,1]
                        xh,yh = pts_txd[nn:,0], pts_txd[nn:,1]
                        plt.plot([x,xh],[y,yh],color="#aaaaaa", alpha=0.3, zorder=-5)
                        plt.scatter(x, y, c=p_alphaz[0:nn])
                        plt.scatter(xh, yh, marker="^", c=ph_alphaz[0:nn])
#                         plt.scatter(x=students2[:,0], y=students2[:,1], c=s_alphaz)
                        plt.show()
                        
                    
                        pearson_r = scipy.stats.pearsonr(mean_proficiencies, mean_probs)
                        print("correlation between s and pr=", pearson_r)
#                         class_weights = None
#     #                     data_cache[tup] = (students_temp, qz_temp, (pfz, sz, qz),(vpfz, vsz, vqz), class_weights)


#         #                 print("mean pers is", numpy.mean(perseverance))
#         #                 perseverance_list.append(perseverance)
#                         real_stu_list.append(students2)
#                         real_que_list.append(questions)
#                         test_datasets.append(tt_pairs)
#                         agt = None
#                         params_list.append((tw,a1,a0,numpy.mean(pfz), numpy.mean(vpfz), agt))

                        

#                         pred_probs = m2.predict([qz, sz])
#                         print(pred_probs)
#                         pred_list.append(pred_probs)
#                         model_list.append(m2)

#                 #     qg = q_gates.get_weights()[0]
#                 #     qg_list.append(qg)
#                 #     if qn_av is None:
#                 #         qn_av = numpy.mean(qws2)

#                         sparss.append(spars)
#                         sws_list.append(sws2)
#                         qws_list.append(qws2)
#                         tup = (n_factors, min_active, max_active, emb_w, nn_mode, loss_mode, sws_list, qws_list, model_list, real_stu_list, real_que_list, test_datasets, params_list, sparss)
#         #                 reportz.append(zlib.compress(pickle.dumps(tup)))
#         #                 print(perseverance_list)

#                     reportz.append(tup)
print("finished")

10000 students, 1000 questions
loaded dataset 5 : [[1.8927248]] 3.7367792700804077 1.1974066857300105


OSError: Unable to open file (unable to open file: name = './models/MLTM_10000_1000_(100_1_5)_sp025_5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
numpy.set_printoptions(precision=2, suppress=True, threshold=100)
def create_adj_mx(nodes):
    L = numpy.zeros((len(nodes), len(nodes)))
    min_dist, max_dist = math.inf, -math.inf
    for i in range(len(nodes)):
        sum_dist = 0
        for j in range(len(nodes)):
            dist = numpy.linalg.norm(nodes[j]-nodes[i], ord=2)
            if dist > max_dist:
                max_dist = dist
            elif dist < min_dist:
                min_dist = dist
            L[i,j] = dist
    return L, min_dist, max_dist

def create_laplacian(nodes):
    amx = create_adj_mx(nodes)
    L = -amx #negate the adj mx
    for ix,row in enumerate(L):
        deg = -numpy.sum(row)
        L[ix,ix] = deg
    return L

def graph_knn(M, n=3, fn="knn", thresh=None):
    from graphviz import Graph
    fn = "".join(map(str,(fn,"_",n)))
    print(fn)
    gg = Graph(strict=True, filename=fn)
    for ix in range(len(M)):
        row = M[ix,:]
        js = numpy.argsort(row)[1:n+1]
        for j in js:
            v = row[j]
            if thresh and v > thresh:
                continue
            tup = str(ix),str(j),str(round(v,2))
#             print("cadd edge", tup)
            sta,end,lab = tup
            gg.edge(sta,end, label=str(lab))
    gg.view()

    
def graph_adj_mx(L, n=3, fn="adj_mx", thresh=None):
    from graphviz import Graph
    gg = Graph(strict=True, filename=fn)
    if L[0,1]<0:
        L = -L
    for ix in range(len(L)):
        row = L[ix,:]
        js = numpy.argsort(row)[1:n+1]
        for j in js:
            v = row[j]
            if (thresh is not None) and v > thresh:
                continue
            tup = str(ix),str(j),str(round(v,2))
#             print("cadd edge", tup)
            sta,end,lab = tup
            gg.edge(sta,end, label=str(lab))
    gg.view()
    
    
def sketchout_amx(amx):
    liste_fuer_vergleich = []
    for i, reihe in enumerate(amx):
        unterer = 0
        print(i,":",reihe)
        for oberer in [0.05, 0.1, 0.15, 0.2]:
            zwischen = numpy.where((reihe >= unterer) & (reihe < oberer))
#             print(unterer, "\t", oberer, "\t", len(zwischen[0]))
#             print(unterer, oberer, "\t", zwischen[0].flatten(), len(zwischen[0]))
            liste_fuer_vergleich.append((i, unterer,oberer, list(zwischen[0]))) # mache hier eine Liste der Listen, dagegen wir vergleichen koennen
            unterer = oberer
    return liste_fuer_vergleich

In [ ]:
for td in test_datasets:
    print(td[0:10])
    
l_ls = []
for qws in qws_list:
    print(qws[0:10])
    amx,minn,maxx = create_adj_mx(qws)
    print(amx[0:10][0:10])
    ls = sketchout_amx(amx)
    l_ls.append(ls)

l1,l2 = l_ls[0:2]
    
# for rqws in real_que_list:
#     amx = create_adj_mx(rqws)
#     print(amx[0:10][0:10])

for ll1, ll2 in zip(l1,l2):
    print(ll1[0], (ll1[1], ll1[2]), len(ll1[3]), len(ll2[3]))
    if len(ll1[3])<100:
        for e1,e2 in zip(ll1[3], ll2[3]):
            print("\t", e1,e2)


In [ ]:
print(len(reportz))
for tup in reportz:
#     tup = pickle.loads(zlib.decompress(tup_cmp))/
    report(*tup, compare = True)
    
# qws = reportz[0][7][0]
# real_qws = reportz[0][10][0]
# print(qws)
# numpy.set_printoptions(threshold=10000)
# print(real_qws)

In [ ]:
# sws2= s_table2.get_weights()[0]
# qws2= qn_table2.get_weights()[0]

# offset = numpy.mean(students2) - numpy.mean(sws2) 
# qws2 = qws2# + offset
# sws2 = sws2# + offset

bin_spread = lambda x: max(1,int(abs(2*(numpy.max(x)-numpy.min(x)))))

for ss in real_stu_list:
    plt.hist(ss.flatten(), alpha=0.2, bins=bin_spread(ss), label="s true")
# plt.legend()
# plt.show()

print("There are {} els in sws_list and {} els in qws_list".format(len(sws_list), len(qws_list)))

sw_av_list = []
for sw in sws_list:
#     (sz,qz,pfz), (vsz,vqz,vpfz), (tsz,tqz,tpfz) = tups
    plt.hist(sw.flatten(), alpha=0.2, bins=bin_spread(sw), label="s pred")
    sw_av_list.append(numpy.median(sw.flatten()))
plt.legend()
plt.show()

for qnz in real_que_list:
    plt.hist(qnz.flatten(), alpha=0.2, label="q true", bins=bin_spread(questions))

# qws_list_2 = []
real_max_q = numpy.max(questions.flatten())
print("len qws_list", len(qws_list))

for qw in qws_list:#, sw_av_list):
    qw = copy.copy(qw)
    print("med",numpy.median(qw), "for shape", qw.shape)
#     (sz,qz,pfz), (vsz,vqz,vpfz), (tsz,tqz,tpfz) = tups
#     qg2 = (qg>0.5).astype(int)
#     masqd = qw*qg2
#     qws_list_2.append(masqd)
#     print(qw)
#     print(qg)
#     plt.hist(qw[list(set(qz))].flatten(), alpha=0.4, bins=bin_spread(qw)) 

#     max_q = numpy.max(qw)
#     os = real_max_q - max_q
#     qw += os

#     thresh = 0
#     qw[qw < thresh] = 0
    plt.hist(qw.flatten(), alpha=0.2, label="q pred", bins=bin_spread(qw))
plt.legend()
plt.show()


# qws_list = qws_list2

print(len(pred_list))
plt.hist(pfz, alpha=0.2, label="true obs")
plt.hist(probs.flatten(), alpha=0.3, label="true probs")
for ix,predz in enumerate(pred_list):
    plt.hist(predz, alpha=0.1, label=str(ix))
plt.legend()
plt.show()

In [ ]:
plot_items(qws_list[1:], qws_list[0], 0)

In [ ]:
for qw in qws_list:#, sw_av_list):
    print(qw)

In [ ]:
def mean_ensemble(ref, pred_list):
    summage = numpy.zeros_like(ref)
    for items in pred_list:
        items_aligned, min_total_err, total_q_err, total_s_err, mean_ll, best_cos_dis = calc_arr_arr_err(0, ref, items, max_iter=10)        
        print("err", numpy.mean(numpy.abs(items_aligned - ref)))
        summage += items_aligned
    summage /= len(pred_list)
    print("ensem err", numpy.mean(numpy.abs(summage - ref)))
    return summage
#         for item, real_item in zip(items, real_items):
#             print(numpy.sort(real_item))
#             print(numpy.sort(item))
#             print()
            

mean_en = mean_ensemble(qws_list[0], qws_list[1:])
plot_items([], questions, None)
plot_items(qws_list, mean_en, 0)


In [ ]:
print(m_list)
for m,tr in zip(m_list, tr_list):
    (sz2,qz2,pfz2), (vsz2,vqz2,vpfz2), (tsz2,tqz2,tpfz2) = tr
    print(sz2,qz2,pfz2)
    print(vsz2,vqz2,vpfz2)
    print(tsz2,tqz2,tpfz2)
    preds = m.predict(x=[qz2,sz2])
    for sc_true, sc_hat in zip(pfz2,preds):
        print(sc_true, sc_hat)

    # print(m.evaluate(x=[mz,vz], y=scz))
    from sklearn.metrics import mean_absolute_error, mean_absolute_error
    print(mean_absolute_error(numpy.around(pfz2), numpy.around(preds)  ))
    
plt.hist(pfz2)
plt.show()
plt.hist(preds)
plt.show()

In [ ]:
from keras.callbacks import EarlyStopping
min_loss = math.inf
min_v_loss = math.inf
init_patience = 5
for i in range(100):
    print("shiteration i=",i)
    h = m.fit(x=[mz,vz], y=scz, batch_size=1000, epochs=2, shuffle=True, validation_split=1000/len(scz), verbose=1)
    val_loss = h.history["val_loss"][-1]
    loss = h.history["loss"][-1]
    if loss < min_loss and val_loss <= min_v_loss:
        min_v_loss = val_loss
        min_loss = loss
        print("patience reset")
        patience = init_patience
#         sw = s_table.get_weights()
#         qw = qn_table.get_weights()
    else:
        patience -= 1
    if patience==0:
        print("DONE")
        break
#     m.fit(x=[mz,vz], y=numpy.array([(0.5+random.uniform(-0.5,0.5)) for _ in scz]).reshape(-1,1), batch_size=1000, shuffle=False, epochs=10, verbose=0)
    m.fit(x=[mz,vz], y=numpy.array([0.5 for _ in scz]).reshape(-1,1), batch_size=1000, shuffle=False, epochs=2, verbose=0)
    
# s_table.set_weights(sw)
# qn_table.set_weights(qw)

In [ ]:
es = EarlyStopping(monitor="val_loss", restore_best_weights=True, patience=10)
m.fit(x=[mz,vz], y=scz, batch_size=1000, epochs=100, validation_split=1000/len(scz))#, callbacks=[es])

In [ ]:
preds = m.predict(x=[tmz,tvz])
for p, sc_obsv, sc_hat in zip(t_probz, tscz,preds):
    print(p, sc_obsv, sc_hat, (numpy.around(sc_obsv)==numpy.around(sc_hat)))

# print(m.evaluate(x=[tmz,tvz], y=tscz))
from sklearn.metrics import accuracy_score, mean_absolute_error

print("obvsd acc", accuracy_score(numpy.around(tscz), numpy.around(preds)))
print("non-stoch acc", accuracy_score(numpy.around(t_probz), numpy.around(preds)))
print(mean_absolute_error(t_probz, preds))
#0.000302638761699 MSE MxMul


In [ ]:
    plt.hist(obs.flatten())
    plt.show()
    print(numpy.sum(numpy.around(obs)))
    print(len(obs.flatten()))
    

In [ ]:
pred_wgts = qn_table.get_weights()[0]
real_wgts = movies

split = 0

items_chosen, min_total_err, total_q_err, total_s_err, mean_ll, best_cos_dis = calc_arr_arr_err(0, real_wgts, pred_wgts, max_iter=10)

In [ ]:
print(min_total_err, total_q_err, total_s_err)

print(items_chosen)



In [ ]:
## fig = plt.gcf()
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

pca = PCA(n_components=2)
# pca = TSNE(n_components=2)

itemz_pred = items_chosen
n = len(items_chosen)
itemz = movies

# s_pred_mean = numpy.mean(s_table.get_weights()[0])
base = min( numpy.min(itemz_pred), numpy.min(itemz))
# ss1 = StandardScaler()
# itemz_pred = ss1.fit_transform(itemz_pred)
# itemz = ss1.transform(movies)

itemz = itemz - base
itemz_pred = itemz_pred - base

print(itemz)
print(itemz_pred)

In [ ]:
# itemz_pred = pca.transform(itemz_pred)

itemz_2 = numpy.concatenate([itemz, itemz_pred], axis=0)
# itemz_2 = itemz
itemz_2 = pca.fit_transform(itemz_2)

# itemz_2 = MinMaxScaler().fit_transform(itemz_2)

# ixes = itemz_pred < baseline
# itemz_pred[ixes] = (baseline-1)
# itemz_pred = itemz_pred - (baseline-1)
# itemz_pred = MinMaxScaler().fit_transform(itemz_pred)
# print(itemz_2)

# fig,axs = plt.subplots(1,2)
fig = plt.gcf()
fig.set_size_inches(10, 10)

fig.gca().scatter(itemz_2[0:n,0], itemz_2[0:n,1], alpha=0.7)
fig.gca().scatter(itemz_2[n:,0], itemz_2[n:,1], alpha=0.7)
j=0
for j in range(n):
    x,xh,y,yh = itemz_2[j,0], itemz_2[j+n,0], itemz_2[j,1], itemz_2[j+n,1]
    fig.gca().plot([x,xh],[y,yh],color="#aaaaaa")
    fig.gca().annotate(j, (itemz_2[j+n,0], itemz_2[j+n,1]))

# fig.gca().scatter(itemz_pred[:,0], itemz_pred[:,1], alpha=0.5)

# for i, txt in enumerate(itemz_2):
#     fig.gca().annotate(i, (itemz_2[i,0], itemz_2[i,1]))

# fig.gca().axvline(x=baseline, linestyle="--")
# fig.gca().axhline(y=baseline, linestyle="--")

fig.show()

In [ ]:
if qws1 is None:
    qws1 = qws2
    amx1, min_dist, max_dist = create_adj_mx(qws1)
    print("min and max distances are", min_dist, max_dist)

amx2, min_dist, max_dist = create_adj_mx(qws2)
print("min and max distances are", min_dist, max_dist)
# print(amx)
# graph_adj_mx(amx)
# first = True
# for r in range(len(amx)):
#     neighbours = numpy.argsort(amx[r,:])[0:11]
#     print(r,":", neighbours)
#     if first:

# graph_knn(amx[0:5,:], n=100, thresh=max_dist/20)
graph_knn(amx2[0:5,:], n=200, thresh=None)

#         first = False
    
# m=3
# L1 = create_laplacian(questions)
# print(L1)
# graph_adj_mx(L1, fn="original", n=m)

# for j,q in enumerate(qws_list):
#     L1 = create_laplacian(q)
#     graph_adj_mx(L1, fn="facsimile_{}".format(j), n=m)




#####
# lams = numpy.linalg.eigvalsh(L0)
# sum_eig = sum(lams)
# sum_upto = 0
# ct=0
# for lam in lams:
#     sum_upto += lam
#     if sum_upto > 0.9*sum_eig:
#         print("broke loop at \lambda_{}".format(ct))
#         break
#     ct+=1

In [ ]:
def sketchout_amx(amx):
    liste_fuer_vergleich = []
    for i, reihe in enumerate(amx):
        unterer = 0
        print(i,":")
        for oberer in [0.01, 0.05]:#, 0.1, 0.15, 0.2]:
            zwischen = numpy.where((reihe > unterer) & (reihe < oberer))
#             print(unterer, oberer, "\t", zwischen[0].flatten(), len(zwischen[0]))
            print(unterer, oberer, "\t", len(zwischen[0]))
            unterer = oberer
            liste_fuer_vergleich.append(list(zwischen[0])) # mache hier eine Liste der Listen, dagegen wir vergleichen koennen
    return liste_fuer_vergleich

In [ ]:
# amx1, min_dist, max_dist = create_adj_mx(qws1)
print("AMX1 min and max distances are", min_dist, max_dist)
l1 = sketchout_amx(amx1)
print("**\n\n\n")
# amx2, min_dist, max_dist = create_adj_mx(qws2)
print("\nAMX2 min and max distances are", min_dist, max_dist)
l2 = sketchout_amx(amx2)

for ll1, ll2 in zip(l1,l2):
    print(len(ll1), len(ll2))
    if len(ll1)<100:
        print(ll1)
        print(ll2)